In [50]:
import warnings
warnings.filterwarnings("ignore")

In [51]:
import pandas as pd
import re
import gensim
from gensim.models import Word2Vec
import numpy as np
from scipy import linalg

In [52]:
data = pd.read_csv("./all_data.csv")
data.head()

,book_title,Book_series,book_series_url,book_image,book_image_url,book_rating,book_author,author_url,genre,reviewer_name,reviewer_url,reviewer_image,reviewer_image_url,review,ID
0,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",emma,https://www.goodreads.com/user/show/32879029-emma,https://images.gr-assets.com/users/1520568130p...,https://www.goodreads.com/user/show/32879029-emma,202.\nThat’s the number of times that the word...,1
1,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Em (RunawayWithDreamthieves),https://www.goodreads.com/user/show/42130592-e...,https://images.gr-assets.com/users/1538507799p...,https://www.goodreads.com/user/show/42130592-e...,I need a black silk robe and a balcony to wear...,2
2,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Cait • A Page with a View,https://www.goodreads.com/user/show/47412532-c...,https://images.gr-assets.com/users/1523134490p...,https://www.goodreads.com/user/show/47412532-c...,THIS BOOK TOTALLY WRECKED ME. In the best poss...,3
3,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Natalie Monroe,https://www.goodreads.com/user/show/6688207-na...,https://images.gr-assets.com/users/1339730817p...,https://www.goodreads.com/user/show/6688207-na...,"3.75 stars\n""Rhys stepped off the stairs and t...",4
4,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Emily May,https://www.goodreads.com/user/show/4622890-em...,https://images.gr-assets.com/users/1528390751p...,https://www.goodreads.com/user/show/4622890-em...,I wish so much that I could tell you I loved t...,5


# Data Preprocessing

In [53]:
data.isnull().sum()

book_title                0
Book_series           12779
book_series_url       12779
book_image                2
book_image_url            2
book_rating               0
book_author               0
author_url                0
genre                   121
reviewer_name            25
reviewer_url             25
reviewer_image           25
reviewer_image_url       25
review                  742
ID                        0
dtype: int64

In [54]:
data.dtypes

book_title             object
Book_series            object
book_series_url        object
book_image             object
book_image_url         object
book_rating           float64
book_author            object
author_url             object
genre                  object
reviewer_name          object
reviewer_url           object
reviewer_image         object
reviewer_image_url     object
review                 object
ID                      int64
dtype: object

In [55]:
data['Book_series'] = data['Book_series'].fillna(data['Book_series'].mode().iloc[0])
data['book_series_url'] = data['book_series_url'].fillna(data['book_series_url'].mode().iloc[0])
data['book_image'] = data['book_image'].fillna(data['book_image'].mode().iloc[0])
data['book_image_url'] = data['book_image_url'].fillna(data['book_image_url'].mode().iloc[0])
data['genre'] = data['genre'].fillna(data['genre'].mode().iloc[0])
data['reviewer_name'] = data['reviewer_name'].fillna(data['reviewer_name'].mode().iloc[0])
data['reviewer_url'] = data['reviewer_url'].fillna(data['reviewer_url'].mode().iloc[0])
data['reviewer_image'] = data['reviewer_image'].fillna(data['reviewer_image'].mode().iloc[0])
data['reviewer_image_url'] = data['reviewer_image_url'].fillna(data['reviewer_image_url'].mode().iloc[0])
data['review'] = data['review'].fillna(data['review'].mode().iloc[0])

In [56]:
data.isnull().sum()

book_title            0
Book_series           0
book_series_url       0
book_image            0
book_image_url        0
book_rating           0
book_author           0
author_url            0
genre                 0
reviewer_name         0
reviewer_url          0
reviewer_image        0
reviewer_image_url    0
review                0
ID                    0
dtype: int64

In [57]:
data.duplicated().sum()

0

In [58]:
data['book_title'].duplicated().sum()

20837

In [59]:
data['book_title'].drop_duplicates(keep=False,inplace=True)
data.head()

,book_title,Book_series,book_series_url,book_image,book_image_url,book_rating,book_author,author_url,genre,reviewer_name,reviewer_url,reviewer_image,reviewer_image_url,review,ID
0,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",emma,https://www.goodreads.com/user/show/32879029-emma,https://images.gr-assets.com/users/1520568130p...,https://www.goodreads.com/user/show/32879029-emma,202.\nThat’s the number of times that the word...,1
1,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Em (RunawayWithDreamthieves),https://www.goodreads.com/user/show/42130592-e...,https://images.gr-assets.com/users/1538507799p...,https://www.goodreads.com/user/show/42130592-e...,I need a black silk robe and a balcony to wear...,2
2,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Cait • A Page with a View,https://www.goodreads.com/user/show/47412532-c...,https://images.gr-assets.com/users/1523134490p...,https://www.goodreads.com/user/show/47412532-c...,THIS BOOK TOTALLY WRECKED ME. In the best poss...,3
3,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Natalie Monroe,https://www.goodreads.com/user/show/6688207-na...,https://images.gr-assets.com/users/1339730817p...,https://www.goodreads.com/user/show/6688207-na...,"3.75 stars\n""Rhys stepped off the stairs and t...",4
4,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Emily May,https://www.goodreads.com/user/show/4622890-em...,https://images.gr-assets.com/users/1528390751p...,https://www.goodreads.com/user/show/4622890-em...,I wish so much that I could tell you I loved t...,5


In [60]:
data["book_rating"]= data["book_rating"].astype(str) 

In [61]:
data['book_title'] = data['book_title'].replace('[^a-zA-Z ]', ' ', regex=True)
data['review'] = data['review'].replace('[^a-zA-Z0-9 ]', ' ', regex=True)

In [62]:
data = data[['book_title', 'review', 'book_rating']]
data = data[pd.notnull(data['review'])]
data.sample(60)

,book_title,review,book_rating
13382,Roman,Yay Another super sweet and safe read I love...,4.1
7495,Long May She Reign,I only bought this purely because a few people...,3.62
4646,The Final Spark,I can t wait till this comes out,4.24
12770,The Romance Reader s Guide to Life,I alway wish you could do half stars I would ...,3.54
429,The Dark Prophecy,WHY YOU NEED TO READ THIS BOOK lesbian warr...,4.26
649,Pachinko,First and foremost Pachinko is an engrossin...,4.26
17096,What the Dead Leave Behind,There is some good potential in this start to ...,3.88
15989,Akata Warrior,This sequel continues to explore the vivid and...,4.27
10217,Matilda Empress,This review has been hidden because it contain...,3.52
10211,Matilda Empress,This review has been hidden because it contain...,3.52


In [63]:
data.shape

(21559, 3)

In [64]:
data.isnull().sum()

book_title     0
review         0
book_rating    0
dtype: int64

In [65]:
data.index = range(21559)
data['review'].apply(lambda x: len(x.split(' '))).sum()

7466388

In [66]:
def print_review(index):
    example = data[data.index == index][['review', 'book_title']].values[0]
    if len(example) > 0:
        print('Review:\n', example[0])
        print('\nBook Title:\n', example[1])
        
print_review(12)

Review:
 When the book you re highly anticipating has no cover  no title  and no synopsis  And a May 2017 release date 

Book Title:
 A Court of Wings and Ruin


In [67]:
print_review(25)

Review:
 FULL REVIEW POSTED If you re reading this  please be warned that you re in for one hell of a rant  There will be quite a lot of swearing and some minor spoilers   Major spoilers will be marked   Also  if you liked this book and have a problem with me trashing everything you ve ever loved  you may want to turn back now    I m giving you one more chance  This is actually the worst book rant I ve ever written    No  You really want to read this review  Move right the fuck along  then  Don t say I didn t warn you  You know when you re at the beach on a really nice summer day  and the sand is all nice and soft and the sun is warm but not too hot and you walk down to the edge of the water for what you just know is going to be a gloriously refreshing swim in the ocean       so you start wading in  and it seems fine at first  and then all of a sudden your foot comes down in the middle of a massive swath of slimy  smelly  warm seaweed aND YOU TRY TO STEP BACK OUT BUT SUDDENLY IT S EVER

# 3. Features Extraction

In [83]:
import nltk
from gensim.models import doc2vec
from gensim.models import Word2Vec
import logging

from gensim.models.doc2vec import (
    Doc2Vec,
    TaggedDocument,
)

In [80]:
import nltk
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens
review = data.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['review']), tags=[r.book_title]), axis=1)

In [73]:
print("Review:\n", tokens.values[30])

NameError: name 'tokens' is not defined

In [70]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [71]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [74]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [75]:
model_review = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_review.build_vocab([x for x in tqdm(review.values)])

NameError: name 'tqdm' is not defined

In [76]:
len(model_review.docvecs)

0

In [77]:
model_review.wv.vocab.keys()

dict_keys([])

In [79]:
model_review.most_similar('spoilers')

KeyError: "word 'spoilers' not in vocabulary"